In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [1]:
KK_PATH = '/content/'
DATASET_PATH = 'code_blocks_prepared.csv' ## CODE_COLUMN = 'code_block'
# DATASET_PATH = 'chunks_30_final.csv' ## CODE_COLUMN = 'code'
# DATASET_PATH = 'nl2ml preprocessing (united).csv' ## CODE_COLUMN = 'Code'
NEW_DATASET_PATH = DATASET_PATH[:-4] + '_regex.csv'
CODE_COLUMN = 'code_block'

### TF-IDF + Log Reg

In [ ]:
df = pd.read_csv(NEW_DATASET_PATH, encoding='utf-8',comment='#', quoting=csv.QUOTE_NONE, error_bad_lines=False)#, sep=','

b'Skipping line 8: expected 10 fields, saw 14\nSkipping line 15: expected 10 fields, saw 12\nSkipping line 23: expected 10 fields, saw 12\nSkipping line 30: expected 10 fields, saw 12\nSkipping line 36: expected 10 fields, saw 12\nSkipping line 44: expected 10 fields, saw 12\nSkipping line 52: expected 10 fields, saw 12\nSkipping line 58: expected 10 fields, saw 11\nSkipping line 76: expected 10 fields, saw 13\nSkipping line 94: expected 10 fields, saw 11\nSkipping line 111: expected 10 fields, saw 11\nSkipping line 130: expected 10 fields, saw 11\nSkipping line 147: expected 10 fields, saw 11\nSkipping line 169: expected 10 fields, saw 12\nSkipping line 229: expected 10 fields, saw 15\nSkipping line 246: expected 10 fields, saw 13\nSkipping line 254: expected 10 fields, saw 15\nSkipping line 269: expected 10 fields, saw 15\nSkipping line 298: expected 10 fields, saw 16\nSkipping line 316: expected 10 fields, saw 18\nSkipping line 333: expected 10 fields, saw 17\nSkipping line 342: exp

In [ ]:
df.head()

,index,Unnamed: 0,code_block,tag,imports,load_data,preprocessing,model,train,predict
0,0,0,"""from mpl_toolkits.mplot3d import Axes3D",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,from sklearn.preprocessing import StandardScaler,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,import matplotlib.pyplot as plt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,import numpy as np,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,import os,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df['preprocessing'].value_counts()

0                                                                                                                                                                                                       117098
1                                                                                                                                                                                                        14406
"                                                                                                                                                                                                          721
                                                                                                                                                                                                           534
 'TotRmsAbvGrd']                                                                                                                                                            

In [ ]:
corpus = df[CODE_COLUMN]
test_size = 0.1
test_rows = round(df.shape[0]*test_size)
train_rows = df.shape[0] - test_rows
train_corpus = df[CODE_COLUMN][0:test_rows]
test_corpus = df[CODE_COLUMN][train_rows:]
tag_to_predict = 'preprocessing'

In [ ]:
tfidf = TfidfVectorizer(min_df=5
                        , max_df = 0.3
                        , ngram_range = (1,2)
                        , smooth_idf = True
                       )
features = tfidf.fit_transform(corpus)
# for_pred = tfidf.transform(test_corpus)

ValueError: ignored

In [ ]:
%time
X_train, X_test, y_train, y_test = train_test_split(features[0:len(df[tag_to_predict])], df[tag_to_predict], test_size=0.25)
clf = LogisticRegression(random_state=421).fit(X_train, y_train)
# clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = clf.score(X_test, y_test)
print(f'Mean Accuracy {round(accuracy*100, 2)}%')
print(f'F1-score {round(metrics.f1_score(y_pred, y_test)*100, 2)}%')

In [ ]:
errors = y_test - y_pred
plt.hist(errors)

In [ ]:
metrics.plot_precision_recall_curve(clf, X_test, y_test)

In [ ]:
metrics.plot_confusion_matrix(clf, X_test, y_test, values_format='d')

In [ ]:
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h
mean_confidence_interval(errors, 0.95)

(0.020153646612790583, 0.017347029322052163, 0.022960263903529003)